In [9]:
import numpy as np
from scipy.optimize import minimize
from cvxopt import matrix, solvers
import os
import sys

In [10]:
#Prereq Stuff
from scipy.spatial.distance import cdist
from matplotlib import pyplot as plt


def linearKernel(X1, X2):
    return X1 @ X2.T


def polyKernel(X1, X2, degree):
    return (X1 @ X2.T + 1) ** degree


def gaussKernel(X1, X2, width):
    distances = cdist(X1, X2, 'sqeuclidean')
    return np.exp(- distances / (2*(width**2)))


def generateData(n, gen_model):

    # Controlling the random seed will give you the same 
    # random numbers every time you generate the data. 
    # The seed controls the internal random number generator (RNG).
    # Different seeds produce different random numbers. 
    # This can be handy if you want reproducible results for debugging.
    # For example, if your code *sometimes* gives you an error, try
    # to find a seed number (0 or others) that produces the error. Then you can
    # debug your code step-by-step because every time you get the same data.

    # np.random.seed(0)  # control randomness when debugging

    if gen_model == 1 or gen_model == 2:
        # Gen 1 & 2
        d = 2
        w_true = np.ones([d, 1])

        X = np.random.randn(n, d)

        if gen_model == 1:
            y = np.sign(X @ w_true)  # generative model 1
        else:
            y = np.sign((X ** 2) @ w_true - 1)  # generative model 2

    elif gen_model == 3:
        # Gen 3
        X, y = generateMoons(n)

    else:
        raise ValueError("Unknown generative model")

    return X, y


def generateMoons(n, noise=0.1):
    n_samples_out = n // 2
    n_samples_in = n - n_samples_out
    outer_circ_x = np.cos(np.linspace(0, np.pi, n_samples_out))
    outer_circ_y = np.sin(np.linspace(0, np.pi, n_samples_out))
    inner_circ_x = 1 - np.cos(np.linspace(0, np.pi, n_samples_in))
    inner_circ_y = 1 - np.sin(np.linspace(0, np.pi, n_samples_in)) - 0.5

    X = np.vstack(
        [np.append(outer_circ_x, inner_circ_x), 
         np.append(outer_circ_y, inner_circ_y)]
    ).T
    X += np.random.randn(*X.shape) * noise

    y = np.hstack(
        [-np.ones(n_samples_out, dtype=np.intp), 
         np.ones(n_samples_in, dtype=np.intp)]
    )[:, None]
    return X, y


def plotPoints(X, y):
    # plot the data points from two classes
    X0 = X[y.flatten() >= 0]
    X1 = X[y.flatten() < 0]

    plt.scatter(X0[:, 0], X0[:, 1], marker='x', label='class -1')
    plt.scatter(X1[:, 0], X1[:, 1], marker='o', label='class +1')
    return


def getRange(X):
    x_min = np.amin(X[:, 0]) - 0.1
    x_max = np.amax(X[:, 0]) + 0.1
    y_min = np.amin(X[:, 1]) - 0.1
    y_max = np.amax(X[:, 1]) + 0.1
    return x_min, x_max, y_min, y_max


def plotModel(X, y, w, w0, classify):

    plotPoints(X, y)

    # plot model
    x_min, x_max, y_min, y_max = getRange(X)
    grid_step = 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, grid_step),
                         np.arange(y_min, y_max, grid_step))
    z = classify(np.c_[xx.ravel(), yy.ravel()], w, w0)

    # Put the result into a color plot
    z = z.reshape(xx.shape)
    plt.contourf(xx, yy, z, cmap=plt.cm.RdBu, alpha=0.5)
    plt.legend()
    plt.show()
    return


def plotAdjModel(X, y, a, a0, kernel_func, adjClassify):

    plotPoints(X, y)

    # plot model
    x_min, x_max, y_min, y_max = getRange(X)
    grid_step = 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, grid_step),
                         np.arange(y_min, y_max, grid_step))
    z = adjClassify(np.c_[xx.ravel(), yy.ravel()], a, a0, X, kernel_func)

    # Put the result into a color plot
    z = z.reshape(xx.shape)
    plt.contourf(xx, yy, z, cmap=plt.cm.RdBu, alpha=0.5)
    plt.legend()
    plt.show()
    return


def plotDualModel(X, y, a, b, lamb, kernel_func, dualClassify):

    plotPoints(X, y)

    # plot model
    x_min, x_max, y_min, y_max = getRange(X)
    grid_step = 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, grid_step),
                         np.arange(y_min, y_max, grid_step))
    z = dualClassify(np.c_[xx.ravel(), yy.ravel()], a, b, X, y, 
                     lamb, kernel_func)

    # Put the result into a color plot
    z = z.reshape(xx.shape)
    plt.contourf(xx, yy, z, cmap=plt.cm.RdBu, alpha=0.5)
    plt.legend()
    plt.show()

    return


def plotDigit(x):
    img = x.reshape((28, 28))
    plt.imshow(img, cmap='gray')
    plt.show()
    return



In [11]:
#q2a

def objective_function(params, y, lamb, K):
    n = len(y)
    
    alpha = params[:n]
    alpha0 = params[n]
    
    linear_combination = K @ alpha + alpha0
    loss = np.sum(np.logaddexp(0, -y * linear_combination))
    
    regularization = (lamb / 2) * np.dot(alpha.T, K @ alpha)
    return loss + regularization


def adjBinDev(X, y, lamb, kernel_func):
    n, d = X.shape
    K = kernel_func(X, X)
    initial_params = np.zeros(n + 1)
    
    result = minimize(objective_function, initial_params, args=(y, lamb, K))
    
    alpha = result.x[:-1]
    alpha0 = result.x[-1]

    return alpha, alpha0



#  usage
X,y = generateData(100,2)
lamb = 0.1  
kernel_func = lambda X1, X2: np.dot(X1, X2.T)

alpha, alpha_0 = adjBinDev(X, y, lamb,kernel_func)

print("Optimized weights:", alpha)
print("Optimized bias:", alpha_0) 

Optimized weights: [-1.04785175e-03  1.49074224e-04 -3.04967483e-04  9.93145999e-05
 -5.96727510e-04  5.62978555e-05 -3.82082893e-04 -6.02120793e-04
  8.91076438e-05 -6.93995027e-04  3.18651023e-04  2.42809758e-04
 -2.94464361e-04  6.81757274e-05 -1.57393699e-04  8.55761955e-05
 -4.56580487e-04  3.11826642e-04  8.33715107e-05  6.64767910e-04
  8.24615170e-05 -2.02923487e-06  6.73003799e-05 -3.97935850e-04
  8.27187727e-04  6.53168494e-04 -9.24504426e-04 -5.50654988e-04
 -4.81893521e-04 -5.90709164e-04  2.48423891e-04  3.35830813e-04
  2.46673844e-04 -3.04464221e-04 -2.41770337e-04  6.10263941e-04
 -1.99835619e-04 -1.58542212e-04 -4.95417276e-04  4.71164237e-04
 -6.09841697e-04 -1.81395168e-04 -4.43910640e-05 -3.34206774e-05
 -2.09517616e-04  1.59832430e-04 -4.34801857e-04  2.12788646e-04
 -2.34617382e-04  6.79733796e-04  1.87587418e-04 -1.00804645e-03
  3.71343446e-04  3.93347308e-05 -5.32320959e-04 -6.06818097e-04
 -1.47840895e-04  5.60524734e-04  1.50557957e-04  5.89178707e-04
 -6.72

In [20]:
#q2b

def adjHinge(X, y, lamb, kernel_func, stabilizer=1e-5):
    n, d = X.shape
    print("n = ", n)
    print("d = ", d)
    K = kernel_func(X, X) 

    y = np.array(y, dtype=np.double)
    print("Shape of y : ", y.shape)


    P = np.zeros((2*n + 1, 2*n+1)) 
    P[:n, :n] = K  # Kernel matrix for alpha terms
    P = P + stabilizer * np.eye(2*n+1)  # Stabilization

    q = np.hstack([np.zeros(n + 1), lamb * np.ones(n)])
    
    # Create G matrix
    # G1: For the non-negativity constraints of the slack variables 
    G1 = np.zeros((n, n + 1 + n))
    print("Shape of G1:", G1.shape)
    G1[:, n+1:] = -np.eye(n)  # G13: -Identity matrix for slack variables
    
    # G2: For the hinge constraints
    G2 = np.zeros((n, n + 1 + n))
    print("Shape of G2:", G2.shape)
    G2[:, :n] = -K @ y[:, np.newaxis].flatten()  # G21: -y * K
    G2[:, n] = -y.flatten()              # G22: -y * Identity_n (affecting α_0)
    G2[:, n+1:] = -np.eye(n)         # G23: Identity matrix for slack variables

    G = np.vstack([G1, G2])  # Stack G1 and G2 to form the full G matrix

    # Create the h vector
    h = np.hstack([np.zeros(n), -np.ones(n)]) 

    # Convert 
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
  
    # Solve the quadratic programming problem
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h)

    # Extract solutions for α and α_0
    alphas = np.array(solution['x'][:n])
    alpha0 = np.array(solution['x'][n])

    return alphas, alpha0

# Example usage
X,y = generateData(100,2)
lamb = 1.0  # Regularization parameter

kernel_func = lambda X1, X2: np.dot(X1, X2.T)

a, a0 = adjHinge(X, y, lamb,kernel_func)

#print("Optimized weights:", a.flatten())
#print("Optimized bias:", a0)

n =  100
d =  2
Shape of y :  (100, 1)
Shape of G1: (100, 201)
Shape of G2: (100, 201)


In [13]:
#q2c
def adjClassify(Xtest, a, a0, X, kernel_func):
    yhat = np.sign( (kernel_func(Xtest,X)@ a + a0)  )




In [17]:
#q2d

def compute_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def predict(X, alphas, alpha0, kernel, X_support):
    K = kernel(X_support, X)
    predictions = K.T.dot(alphas) + alpha0
    return np.sign(predictions)

def sunExperimentsKernel():
    n_runs = 10 
    n_train = 100
    n_test = 1000
    lamb = 0.001
    kernel_list = [linearKernel,
                   lambda X1, X2: polyKernel(X1, X2, 2),
                   lambda X1, X2: polyKernel(X1, X2, 3),
                   lambda X1, X2: gaussKernel(X1, X2, 1.0),
                   lambda X1, X2: gaussKernel(X1, X2, 0.5)]
    
    gen_model_list = [1, 2, 3]
    
    train_acc_bindev = np.zeros([len(kernel_list), len(gen_model_list), n_runs])
    test_acc_bindev = np.zeros([len(kernel_list), len(gen_model_list), n_runs])
    train_acc_hinge = np.zeros([len(kernel_list), len(gen_model_list), n_runs])
    test_acc_hinge = np.zeros([len(kernel_list), len(gen_model_list), n_runs])
    
    for r in range(n_runs):
        for i, kernel in enumerate(kernel_list):
            for j, gen_model in enumerate(gen_model_list):
                Xtrain, ytrain = generateData(n=n_train, gen_model=gen_model)
                Xtest, ytest = generateData(n=n_test, gen_model=gen_model)
                
                # BinDev model
                a, a0 = adjBinDev(Xtrain, ytrain, lamb, kernel)
                train_pred_bindev = predict(Xtrain, a, a0, kernel, Xtrain)
                test_pred_bindev = predict(Xtest, a, a0, kernel, Xtrain)
                train_acc_bindev[i, j, r] = compute_accuracy(ytrain, train_pred_bindev)
                test_acc_bindev[i, j, r] = compute_accuracy(ytest, test_pred_bindev)
                
                # Hinge model
                a, a0 = adjHinge(Xtrain, ytrain, lamb, kernel)
                train_pred_hinge = predict(Xtrain, a, a0, kernel, Xtrain)
                test_pred_hinge = predict(Xtest, a, a0, kernel, Xtrain)
                train_acc_hinge[i, j, r] = compute_accuracy(ytrain, train_pred_hinge)
                test_acc_hinge[i, j, r] = compute_accuracy(ytest, test_pred_hinge)

    # Compute average accuracies over runs
    avg_train_acc_bindev = np.mean(train_acc_bindev, axis=2)
    avg_test_acc_bindev = np.mean(test_acc_bindev, axis=2)
    avg_train_acc_hinge = np.mean(train_acc_hinge, axis=2)
    avg_test_acc_hinge = np.mean(test_acc_hinge, axis=2)
    
    # Return or print results
    return avg_train_acc_bindev, avg_test_acc_bindev, avg_train_acc_hinge, avg_test_acc_hinge

# Call function
avg_train_acc_bindev, avg_test_acc_bindev, avg_train_acc_hinge, avg_test_acc_hinge = sunExperimentsKernel()

# Print results
print("Average Train Accuracies - Binary Deviance:", avg_train_acc_bindev)
print("Average Test Accuracies - Binary Deviance:", avg_test_acc_bindev)
print("Average Train Accuracies - Hinge:", avg_train_acc_hinge)
print("Average Test Accuracies - Hinge:", avg_test_acc_hinge)




ValueError: Rank(A) < p or Rank([P; A; G]) < n